In [1]:
import numpy as np, pandas as pd
rng = np.random.default_rng(7)

In [2]:
plants = pd.DataFrame([
    ("FRE","Fremont","USA",37.5485,-121.9886),
    ("AUS","Austin","USA",30.2672,-97.7431),
    ("BER","Berlin","DE",52.52,13.405),
    ("SHA","Shanghai","CN",31.2304,121.4737)
], columns=["plant_id","plant_name","country","lat","lon"])

In [3]:
regions = pd.DataFrame([("EC","East Coast"),("CC","Central"),("WC","West Coast")], columns=["region_id","region_name"])
models  = pd.DataFrame([("M3","Model 3"),("MY","Model Y")], columns=["model_code","model_name"])
trims   = pd.DataFrame([("ST","Standard"),("PR","Performance")], columns=["trim_code","trim_name"])
colors  = pd.DataFrame([("BK","Black"),("WH","White"),("GY","Gray"),("BL","Blue")], columns=["color_code","color_name"])

In [ ]:
plants.to_csv("plants.csv", index=False)
regions.to_csv("regions.csv", index=False)
models.to_csv("models.csv", index=False)
trims.to_csv("trims.csv", index=False)
colors.to_csv("colors.csv", index=False)

In [ ]:
dates = pd.date_range("2025-07-01","2025-09-30",freq="D")
prod_rows = []
for d in dates:
    for _,p in plants.iterrows():
        daily = rng.integers(120, 240)  # plant output per day
        mix = rng.multinomial(daily, [0.6,0.4])  # M3/MY split
        for m_idx, m in enumerate(["M3","MY"]):
            # trim & color splits
            trim_split = rng.multinomial(mix[m_idx], [0.7,0.3])
            for t_idx,t in enumerate(["ST","PR"]):
                col_split = rng.multinomial(trim_split[t_idx], [0.35,0.35,0.2,0.1])
                for c_idx,c in enumerate(["BK","WH","GY","BL"]):
                    q = int(col_split[c_idx])
                    if q>0:
                        prod_rows.append([p.plant_id,m,t,c,str(d.date()),q])
vehicle_production = pd.DataFrame(prod_rows, columns=["plant_id","model_code","trim_code","color_code","prod_date","qty"])
vehicle_production.to_csv("vehicle_production.csv", index=False)

In [ ]:
weeks = pd.date_range("2025-07-07","2025-09-29",freq="W-MON")
dem_rows=[]
for w in weeks:
    for r in ["EC","CC","WC"]:
        base = {"EC":0.45,"CC":0.30,"WC":0.25}[r]
        for m in ["M3","MY"]:
            for t in ["ST","PR"]:
                for c in ["BK","WH","GY","BL"]:
                    mean = 600*base * (1.1 if m=="MY" else 0.9) * (1.15 if t=="PR" else 1.0)
                    qty = int(max(0, rng.normal(mean, mean*0.15)))
                    if qty>0:
                        dem_rows.append([r,m,t,c,str(w.date()),qty])
regional_demand = pd.DataFrame(dem_rows, columns=["region_id","model_code","trim_code","color_code","week_start","forecast_qty"])
regional_demand.to_csv("regional_demand.csv", index=False)


In [ ]:
tt = []
for p in plants.plant_id:
    for r in regions.region_id:
        base = {"EC": 6, "CC": 5, "WC": 4}[r]

        if p == "FRE":
            base += {"EC": 6, "CC": 4, "WC": 0}[r] 
        elif p == "AUS":
            base += {"EC": 4, "CC": 0, "WC": 5}[r]
        elif p == "BER":
            base += 20
        elif p == "SHA":
            base += 18

        days = base
        cost = 400 + 60 * days

        tt.append([p, r, days, cost])

transit_time = pd.DataFrame(tt, columns=["plant_id","region_id","days_lead_time","per_unit_ship_cost"])
transit_time.to_csv("transit_time.csv", index=False)